IMDB Movies Reviews SENTIMENT ANALYSIS

In [2]:
import numpy as np
import pandas as pd

loading dataset 

In [6]:
data= pd.read_csv("dataset/IMDB Dataset.csv")

print first five rows of dataset

In [ ]:
data.head()

check if there is any null value

In [13]:
data.isnull().any()

review       False
sentiment    False
dtype: bool